<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 100; padding-top: 22px; text-align:right;">Jokin Cuesta Arrillaga</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# IA: DETECTOR DE CANSANCIO CON MEDIAPIPE Y OPENCV


**Importante:** este proyecto se ha realizado en base al vídeo: *AI Body Language Decoder with MediaPipe and Python in 90 Minutes* de Nicholas Renotte. El proyecto se ha llevado a cabo con el único fin de desarrollar habilidades y en **ningún caso ha sido para obtener ningún beneficio económico.**

>A lo largo de este proyecto se utilizará  Mediapipe para estimar puntos de referencia tanto faciales como corporales, obtenidos directamente de una WebCam a través de la OpenCV . Con esos datos, se crearán modelos personalizados de clasificación de poses que le permitan decodificar lo que una persona podría estar diciendo con su lenguaje corporal con gran precisión, adaptando a las necesidades de cada aplicación. En este proyecto en concreto, se tratará de realizar detección de somnolencia para conductores.

---


**Antes de nada, instalaremos todas las dependencias que necesitaremos en este proyecto. Esta es la lista con las librerías necesarias:**

 - **MediaPipe:** ofrece soluciones de ML como el reconocimiento facial, mallas faciales, holístico, poses, detección de objetos, Motion Tracking etc. Para nuestra propuesta, utilizaremos el holístico, lo que nos permitirá reconocer en vivo de la pose humana simultánea , los puntos de referencia faciales y el seguimiento de mano. Obtendremos puntos coordenadas numéricas de articulaciones los cuales guardaremos en un .csv para su posterior tratamiento. Se puede apreciar mejor en este .gif obtenido de https://google.github.io/mediapipe/solutions/holistic.html.
 
<div style="width: 100%; clear: both;">
<div style="float: center; width: 100%;">
<img src="https://google.github.io/mediapipe/images/mobile/holistic_sports_and_gestures_example.gif", align="center">

 - **OpenCV:** Lo utilizaremos para obtener, renderizar y procesar la imagen de la cámara.
 - **Pandas:** Para el tratamiento de los datos obtenidos.
 - **Scikit-learn:** Para llevar adelante todos los modelos de ML.
 

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

>Nota: a día 20/6/2022, existe un problema con las versiones de protobuf cuando se quiere cargar mediapipe. Para solucionarlo, se debe cargar la versión anterior con *!pip install --upgrade protobuf==3.20.1*.

In [2]:
#!pip install --upgrade protobuf==3.20.1
import mediapipe as mp    #Importamos mediapipe
import cv2                #Importamos opencv

In [3]:
mp_drawing = mp.solutions.drawing_utils   #Ayudas para dibujar
mp_holistic = mp.solutions.holistic       #Soluciones de Mediapipe

<a id="ej1"></a>

## 1. REALIZAR ALGUNAS DETECCIONES

En este apartado se realizarán algunas detecciones para la comprobación del correcto funcionamiento.

In [4]:
# Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()      
        
        # Reformatear: BGR -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el modelo holistico:
        results = holistic.process(image)               
        
        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Mostramos la imagen                
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<div class="alert alert-block alert-info">
Se observa en la imagen siguiente que la detección es correcta.
    
---
    
 
    
![imagen de holistic](https://i.ibb.co/bRwV3C3/holistic.png)

---
Ahora, todas las landmarks se guardan en results. Si se escribe directamente el valor de results, obtendremos '*mediapipe.python.solution_base.SolutionOutputs*'. Pero, results está construido por 4 modelos: face_landmarks, right_hand_landmarks, left_hand_landmarks y pose landmarks. 

    
    
---
    
Todos los landmarks del pose que detecta MediaPipe (33) se recogen en la siguiente imagen:
<div style="width: 100%; clear: both;">
<div style="float: center; width: 100%;">
<img src="https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png", align="center">

---
    
Ahora, si compilamos *results.face_landmarks.landmark[0]*, obtendremos las coordenadas en 3D de la nariz:

In [5]:
results.face_landmarks.landmark[0]

x: 0.5337157845497131
y: 0.7088602781295776
z: -0.036886997520923615

<div class="alert alert-block alert-info">
Y podemos obtener los valores x,y,z:

In [6]:
print("x:", results.pose_landmarks.landmark[0].x,"y:", results.pose_landmarks.landmark[0].y,"z:", results.pose_landmarks.landmark[0].z)

x: 0.5251173973083496 y: 0.6522732377052307 z: -1.2449032068252563


<div class="alert alert-block alert-info">
Ahora pasaremos al siguiente apartado para almacenar los datos obtenidos de results en un .csv

----

<a id="ej2"></a>

## 2. CAPTURAR LANDMARKS Y EXPORTAR A ARCHIVO CSV


Importaremos 3 librerías para realizar el almacenamiento de datos.
 - **csv:** nos permitirá trabajar con archivos .csv y escribir nuestros datos en ellos.
 - **os:** usaremos para guardar los archivos en una carpeta específica. 
 - **numpy:** utilizaremos numpy arrays para procesar la información.

In [7]:
#Importamos librerías necesarias:
import csv
import os
import numpy as np

<div class="alert alert-block alert-info">
Queremos cuatro valores de los landmarks: x,y,z,v; siendo v la visibilidad.  El modelo facial no dispone del valor de visibilidad, por lo que para todos los puntos de la cara tendremos un v con valor 0.
    
---
    
Vamos a loopear por todos los landmarks, por lo que nos interesa saber cuántos son. Ya hemos visto que para el pose teníamos 33 landmarks. Ahora nos faltará los de la cara.
</div>

In [8]:
#Tenemos 501 coordenadas en total
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

<div class="alert alert-block alert-info">
Clasificaremos los coordenadas de cada landmark con su número de identificación.
</div>

In [9]:
#la primera columna servirá de identificación de clase 
landmarks = ['class']

for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

#Mostramos ejemplo:
print(landmarks[0:13])

['class', 'x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', 'x3', 'y3', 'z3', 'v3']


<div class="alert alert-block alert-info">
Ahora empezaremos con la exportación al archivo .csv. Escribiremos los nombres de las columnas.
</div>

In [10]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

<div class="alert alert-block alert-info">
Ahora, obtenemos un .csv (en la misma carpeta de este archivo) en el cual tenemos las columnas nombradas. 
    
---
    
Vamos ahora a recopilar datos. Para ello, necesitamos dos clases, el de cansado y despierto. Para cada clase, recogeremos los datos desde la cámara.

In [72]:
class_name = "Cansado"

In [73]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<a id="ej3"></a>

## 3. ENTRENAR MODELO UTILIZANDO Scikit Learn

### 3.1 CARGAR DATOS CON PANDAS

Ahora necesitamos otras dos librerías para entrenar el modelo.
 - Pandas: para el procesamiento del dataframe (df)
 - Sklearn: en concreto *train_test_split*, con el cual dividiremos el dataset en dos: uno para el entrenamiento y el segundo para validar los modelos. 

In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [82]:
# Leemos el archivo .csv
df = pd.read_csv("coords.csv")
df

,class,0.5925989747047424,0.5884896516799927,-1.2507920265197754,0.9995885491371155,0.6283556818962097,0.5241056680679321,-1.18904447555542,0.9990204572677612,0.6487206816673279,...,-0.002297681989148259,0.0.465,0.6713571548461914,0.5337255001068115,0.017656950280070305,0.0.466,0.677161693572998,0.5292463898658752,0.018023835495114326,0.0.467
0,Despierto,0.590247,0.585625,-1.309817,0.999558,0.626095,0.520771,-1.242184,0.998981,0.647467,...,-0.003721,0.0,0.670170,0.533347,0.017821,0.0,0.675708,0.529805,0.018240,0.0
1,Despierto,0.588716,0.583794,-1.351447,0.999538,0.624573,0.518546,-1.280802,0.998957,0.646577,...,-0.002340,0.0,0.669821,0.532206,0.018979,0.0,0.675474,0.528500,0.019327,0.0
2,Despierto,0.587841,0.582345,-1.339246,0.999519,0.623477,0.516835,-1.270343,0.998940,0.645889,...,-0.002133,0.0,0.668993,0.532249,0.018921,0.0,0.674627,0.528241,0.019299,0.0
3,Despierto,0.586610,0.581507,-1.354581,0.999478,0.622621,0.515724,-1.286509,0.998871,0.645388,...,-0.002080,0.0,0.668545,0.532245,0.019440,0.0,0.674098,0.528355,0.019876,0.0
4,Despierto,0.586083,0.580998,-1.367312,0.999493,0.622170,0.514995,-1.300065,0.998895,0.645067,...,-0.002755,0.0,0.668536,0.532499,0.018190,0.0,0.674081,0.528782,0.018540,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,Cansado,0.528294,0.612086,-1.301606,0.998209,0.569529,0.530228,-1.252487,0.996434,0.594966,...,-0.015079,0.0,0.615112,0.505994,0.004764,0.0,0.621191,0.501128,0.004500,0.0
859,Cansado,0.518735,0.613739,-1.328865,0.998278,0.561273,0.530614,-1.279767,0.996530,0.586753,...,-0.015628,0.0,0.605091,0.501498,0.001755,0.0,0.611227,0.496695,0.001264,0.0
860,Cansado,0.511096,0.614515,-1.305153,0.998371,0.555154,0.530650,-1.258515,0.996678,0.580809,...,-0.015079,0.0,0.602980,0.501220,0.002336,0.0,0.609186,0.496648,0.001854,0.0
861,Cansado,0.509096,0.614736,-1.275277,0.998452,0.553073,0.530675,-1.228548,0.996814,0.578445,...,-0.015052,0.0,0.600514,0.498998,0.001501,0.0,0.606683,0.494332,0.000951,0.0


In [83]:
# Guardaremos las variables en X y el target en Y
X = df.drop('class',axis=1)
y = df['class']

In [84]:
# Dividimos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1000)

### 3.2 MODELO DE CLASIFICACIÓN

Para crear el modelo, necesitaremos importar varias librerías. Cabe mencionar que, para que no dependamos únicamente de un modelo de ML, se va a realizar 4 pipelines de modelos. 
 - **sklearn.pipeline:** nos permitirá llevar a cabo esos 4 pipelines.
 - **sklearn.preprocessing:** Para la estandarización.
 - **sklearn.linear_model:** Para importar los modelos lineales *LogisticRegression* y *RidgeClassifier*.
 - **sklearn.ensemble:** Para importar modelos de ensemble como *RandomForestClassifier* y *GradientBoostingClassifier*.

In [85]:
#Importamos las funciones necesarias
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

Realizaremos 4 modelos de ML para no depender únicamente de una.
 - Regresión Logística
 - Ridge Classifier
 - Random Forest Classifier
 - Gradient Boosting Classifier

In [86]:
# Definimos los cuatro pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [87]:
# Guardamos los modelos en la biblioteca fit_models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\Jokin Cuesta\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [88]:
# Probamos a predecir X_test con el modelo de Random Forest
fit_models['rf'].predict(X_test)

array(['Cansado', 'Despierto', 'Despierto', 'Cansado', 'Despierto',
       'Despierto', 'Cansado', 'Cansado', 'Cansado', 'Despierto',
       'Cansado', 'Cansado', 'Despierto', 'Cansado', 'Cansado', 'Cansado',
       'Cansado', 'Cansado', 'Despierto', 'Despierto', 'Despierto',
       'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Despierto', 'Cansado', 'Cansado', 'Cansado',
       'Cansado', 'Cansado', 'Despierto', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Cansado', 'Cansado', 'Cansado', 'Despierto', 'Cansado',
       'Despierto', 'Despierto', 'Cansado', 'Despierto', 'Despierto',
       'Despierto', 'Cansado', 'Cansado', 'Despierto', 'Cansado',
       'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Despierto',
       'Cansado', 'Despierto', 'Cansado', 'Despierto', 'Despierto',
       'Cansado', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Despierto', 'Cansado', 'Despierto',

### 3.3 EVALUACIÓN Y SERIALIZACIÓN DEL MODELO

Ahora es hora de evaluar los modelos construidos. Para ello importaremos:
- **sklearn.metrics:** la función *accuracy_score* para obtener la precisión obtenida de cada modelo al predecir los *targets* de X_test.
- **pickle:** para la serialización del modelo.


In [89]:
from sklearn.metrics import accuracy_score
import pickle

<div class="alert alert-block alert-info">
A continuación se predicen las clases de X_test para cada modelo.

In [90]:
#Predecir las clases de X_test
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    
    #Mostramos las precisiones de cada modelo
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 0.9961389961389961
gb 0.9922779922779923


<div class="alert alert-block alert-info">
Se ve que para los primeros dos (*LogisticRegression*, *RidgeClassifier*), la precisión ha sido del 100%. Para *GradientBoostingClassifier* y *RandomForestClassifier* 99.6%. Por elección propia, utilizaré el *RandomForestClassifier*.

---

Se serializa entonces el modelo obtenido en un archivo .pkl:

In [91]:
# Escribimos el modelo en un archivo .pkl
with open('cansado_despierto.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4.  DETECCIONES Y ALERTA AL USUARIO
<div class="alert alert-block alert-info">
Vamos a ir a la parte final de este proyecto. Realizaremos las detecciones de cansado o despierto con el modelo entrenado. Para ello, volvemos a abrir el modelo serializado.

In [92]:
# Abrimos de nuevo el modelo
with open('cansado_despierto.pkl', 'rb') as f:
    model = pickle.load(f)

<div class="alert alert-block alert-info">
De nuevo se abrirá la cámara y con la ayuda de MediaPipe haremos detecciones de los landmarks. El modelo será capaz de predecir el estado del/la usuario/a con esos datos. En la pantalla, vamos a mostrar la clase predicha y su probabilidad.

---

Cuando la probabilidad de estar cansado supere el 70%, vamos a alertar al usuario que necesita un descanso en un rectángulo rojo, como símbolo de alerta.

In [94]:
#Conectamos la cámara
cap = cv2.VideoCapture(0)

# Inicia el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        
        #Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # BGR -> RGB y seguridad de la imagen.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Detecciones
        results = holistic.process(image)
        
        # RGB -> BGR y seguridad de la imagen.        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Exporta coordenadas
        try:
            # Extraemos Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extraemos Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concatenamos las filas
            row = pose_row+face_row

            # Hacemos detecciones
            X = pd.DataFrame([row])
            
            # Predecir la clase
            body_language_class = model.predict(X)[0]     
            
            # Predecir la probabilidad de la clase
            body_language_prob = model.predict_proba(X)[0]
            
            # Recogemos coordenadas de la oreja izquierda para visualizar texto dinámicamente
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            
            #Mostramos la clase en la oreja izq.
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Status Box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Mostramos Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Mostramos Probabilidad
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Al estar muy cansado, activamos la alerta de descanso.
            if (round(body_language_prob[np.argmax(body_language_prob)],2) > 0.70) and (body_language_class.split(' ')[0] == "Cansado"):
                
                cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len('MUY CANSADO')*20, coords[1]-30), 
                          (0, 0, 255), -1)
                
                cv2.putText(image, 'MUY CANSADO', coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.rectangle(image, (0,0), (550, 60), (0, 0, 255), -1)
                
                cv2.putText(image, "ALERTA! NECESITAS DESCANSAR", (15,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)               

            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 5. CONCLUSIONES y PUNTOS A MEJORAR

En este proyecto hemos realizado un detector de cansancio, como aplicación de seguridad para los/as conductores en la carretera. Los resultados obtenidos han sido satisfactorios (se muestran en el siguiente .gif) y podrían mejorarse teniendo más datos entrenados o tomando en cuenta más variables en cuanto al cansancio. Incluso se podría estudiar más clases donde añadiríamos distintos grados de cansancio.

![imagen de gif ](https://i.ibb.co/pfYBY9H/Cansadodespierto.gif)